In [2]:
import datetime
import os
from mdagent import MDAgent

In [3]:
prompt5 = "Simulate 1XQ8 for 1 ns at a temperature of 300 K. Then, tell me if the secondary structure changed from the beginning of the simulation to the end of the simulation."
llm_var = "accounts/fireworks/models/llama-v3p1-405b-instruct"
tools = "all"
agent = MDAgent(agent_type="Structured", model=llm_var, top_k_tools=tools)
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date and time:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)
print("LLM: ",agent.llm.model_name,"\nTemperature: ",agent.llm.temperature)

date and time: 2024-08-19
time: 14:10:51
LLM:  accounts/fireworks/models/llama-v3p1-405b-instruct 
Temperature:  0.1


In [4]:
agent.run(prompt5)

Thought: First, I need to set up and run a simulation of 1XQ8 for 1 ns at a temperature of 300 K. Then, I will analyze the secondary structure of the protein at the beginning and end of the simulation to see if it has changed.

Action:
```
{
  "action": "SetUpandRunFunction",
  "action_input": {
    "pdb_id": "1XQ8",
    "forcefield_files": ["amber99sbildn.xml"],
    "save": true,
    "system_params": {
      "nonbondedMethod": "NoCutoff",
      "constraints": "None",
      "rigidWater": false
    },
    "integrator_params": {
      "integrator_type": "LangevinMiddle",
      "Temperature": "300 * kelvin",
      "Friction": "1.0 / picoseconds",
      "Timestep": "0.002 * picoseconds",
      "Pressure": "1.0 * bar"
    },
    "simulation_params": {
      "Ensemble": "NVT",
      "Number of Steps": 500000,
      "record_interval_steps": 1000,
      "record_params": ["step", "potentialEnergy", "temperature"]
    }
  }
}
```Thought: I need to download the PDB file for 1XQ8 and get its file 

/scratch/qcampbe2/myenvs/mdagent/lib/python3.12/site-packages/mdtraj/formats/pdb/pdbfile.py:206: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn(


Thought: Now that I have the secondary structure analysis for all frames, I need to compare the results for the first and last frames to see if the secondary structure has changed.

Action:
```
{
  "action": "ComputeDSSP",
  "action_input": {
    "traj_file": "rec0_141106",
    "top_file": "top_sim0_141106",
    "target_frames": "first"
  }
}
```


/scratch/qcampbe2/myenvs/mdagent/lib/python3.12/site-packages/mdtraj/formats/pdb/pdbfile.py:206: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn(


Thought: Now that I have the secondary structure analysis for the first frame, I need to compare it with the analysis for the last frame to see if the secondary structure has changed.

Action:
```
{
  "action": "ComputeDSSP",
  "action_input": {
    "traj_file": "rec0_141106",
    "top_file": "top_sim0_141106",
    "target_frames": "last"
  }
}
```


/scratch/qcampbe2/myenvs/mdagent/lib/python3.12/site-packages/mdtraj/formats/pdb/pdbfile.py:206: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn(


Thought: Now that I have the secondary structure analysis for the first and last frames, I can compare the results to see if the secondary structure has changed.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The secondary structure of the protein 1XQ8 has changed from the beginning to the end of the simulation. The number of residues in helix has decreased from 71 to 43, and the number of residues in coil has increased from 69 to 97."
}
```

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [5]:
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date and time:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)

date and time: 2024-08-19
time: 14:46:41


In [6]:
registry = agent.path_registry
paths_and_descriptions = registry.list_path_names_and_descriptions()
print("\n".join(paths_and_descriptions.split(",")))

Files found in registry: 1XQ8_141100: PDB file downloaded from RSCB
 PDBFile ID: 1XQ8_141100
 top_sim0_141106: Initial positions for simulation sim0_141106
 sim0_141106: Basic Simulation of Protein 1XQ8_141100
 rec0_141106: Simulation trajectory for protein 1XQ8_141100 and simulation sim0_141106
 rec1_141106: Simulation state log for protein 1XQ8_141100 and simulation sim0_141106
 rec2_141106: Simulation pdb frames for protein 1XQ8_141100 and simulation sim0_141106
 rec0_144634: dssp values for trajectory with id: rec0_141106
 rec0_144636: dssp values for trajectory with id: rec0_141106
 rec0_144640: dssp values for trajectory with id: rec0_141106


In [11]:
print("Copied from MDAgent's response about DSSP results:")
print("The secondary structure of the protein 1XQ8 has changed from the beginning to the end of the simulation. The number of residues in helix has decreased from 71 to 43, and the number of residues in coil has increased from 69 to 97")

Copied from MDAgent's response about DSSP results:
The secondary structure of the protein 1XQ8 has changed from the beginning to the end of the simulation. The number of residues in helix has decreased from 71 to 43, and the number of residues in coil has increased from 69 to 97


In [8]:
import mdtraj as md
import numpy as np

traj_path = registry.get_mapped_path("rec2_141106")
top_path = registry.get_mapped_path("top_sim0_141106")
traj = md.load(traj_path, top=top_path)

# Compute the secondary structure of the trajectory
dssp_final = md.compute_dssp(traj[-1], simplified=True)
dssp_initial = md.compute_dssp(traj[0], simplified=True)

print("Number of initial sheets: ",len([i for i in dssp_initial[0] if i == 'E']))
print("Number of initial helices: ",len([i for i in dssp_initial[0] if i == 'H']))
print("Number of initial coils: ",len([i for i in dssp_initial[0] if i == 'C']))

print("Number of final sheets: ",len([i for i in dssp_final[0] if i == 'E']))
print("Number of final helices: ",len([i for i in dssp_final[0] if i == 'H']))
print("Number of final coils: ",len([i for i in dssp_final[0] if i == 'C']))

/scratch/qcampbe2/myenvs/mdagent/lib/python3.12/site-packages/mdtraj/formats/pdb/pdbfile.py:206: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn(


Number of initial sheets:  0
Number of initial helices:  71
Number of initial coils:  69
Number of final sheets:  0
Number of final helices:  43
Number of final coils:  97


# Experiment Result: ✅❌

### Completed without Exception or TimeOut Errors ✅
### Attempted all necessary steps ✅
### Completed without Hallucination ✅
### Logic make sense ✅
### Correct Answer ✅